# GOOGLE 과 10 folds의 결합

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import pickle
import torch.nn.functional as nnf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
import copy
from torchvision.models import googlenet, GoogLeNet_Weights
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. 공통 함수 정의 및 데이터 불러오기

In [ ]:
# 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        # self.label_mapping = label_mapping

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # PKL 파일에서 이미지와 레이블을 가져옴
        img, label = self.data[idx]

        label -= 1
        # 이미지를 PIL 이미지로 변환
        img = transforms.ToPILImage()(img) #PKL에서 가져온 data : tensor --> PIL

        # 이미지에 정의된 변환을 적용
        if self.transform:
            img = self.transform(img)

        return img, label

# transofrm 정의
rotation_range = (-15, 15)
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.RandomPerspective(distortion_scale=.10, p=.15, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.RandomRotation(degrees=10, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.Normalize(mean=[0.5878, 0.5398, 0.4853],std=[0.1505, 0.1592, 0.1703])
])

flip_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.RandomHorizontalFlip(p=1),
     transforms.RandomPerspective(distortion_scale=.10, p=.15, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.RandomRotation(degrees=10, interpolation=transforms.InterpolationMode.NEAREST),
     transforms.Normalize(mean=[0.5878, 0.5398, 0.4853],std=[0.1505, 0.1592, 0.1703])
])

sub_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5915, 0.5468, 0.4935],
                             std=[0.1549, 0.1642, 0.1756]),
    ]
)

# calculate_loss 함수 설정
def calculate_loss(loader, model, criterion):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels) # tensor
            total_loss += loss.item()
    return total_loss / len(loader)

# 데이터 로딩 함수
def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

train_data_vanila = load_pickle('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/train_data_list.pkl')
test_data_vanila = load_pickle('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/test_data_list.pkl')

train_data_contour = load_pickle('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/train_data_contour.pkl')
test_data_contour = load_pickle('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/test_data_contour.pkl')

train_data = train_data_vanila + train_data_contour
test_data = test_data_vanila

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
len(train_data), len(test_data)

(1416, 329)

#### 1.1 파라미터 및 데이터 설정



In [ ]:
in_chnnels = 3
num_classes = 10
learning_rate = 1e-5
batch_size = 8
num_epochs = 200
load_model = False

##
model = torchvision.models.googlenet(init_weights=True)
num_ftrs = model.fc.in_features  # 마지막 계층의 입력 특성 수를 가져옴
model.fc = nn.Linear(num_ftrs, num_classes)  # 새로운 완전 연결 계층으로 대체
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3, verbose=True)

In [ ]:
# 데이터셋 인스턴스 생성

train_dataset_original = CustomDataset(train_data, transform=transform)
train_dataset_filp = CustomDataset(train_data, transform=flip_transform)
train_dataset = torch.utils.data.ConcatDataset([train_dataset_original, train_dataset_filp])

# 테스트 데이터셋 인스턴스 생성
test_dataset = CustomDataset(test_data, transform=sub_transform)

# 데이터셋을 학습용과 검증용으로 분할
train_size = int(0.6 * len(train_dataset))
val_size = int(0.2 * len(train_dataset))
test1_size = len(train_dataset) - train_size - val_size
train_dataset, val_dataset, test1_dataset = random_split(train_dataset, [train_size, val_size, test1_size])

# 분할된 데이터셋을 DataLoader에 로드
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test1_loader = DataLoader(test1_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
len(train_dataset), len(val_dataset), len(test1_dataset), len(test_dataset)

(1699, 566, 567, 329)

#### 1.2 학습 및 평가 함수 정의

In [ ]:
# 모델 학습 함수
def train_model(model, train_loader, criterion, optimizer, num_epochs, scheduler):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0 # 매 에폭에서 손실 초기화
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs, aux1, aux2 = model(images)
            # 주 출력, 보조 출력에 대한 손실 계산
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux1, labels)
            loss3 = criterion(aux2, labels)
            loss = loss1 + 0.3 * (loss2)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() # 현재 배치의 손실 누적 (기존 losses변수 --> running_loss)

        mean_loss = running_loss/len(train_loader)
        scheduler.step(mean_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")


# 모델 전체 정확도, 라벨별 정확도 계산 함수
def evaluate_and_calculate_accuracy(model, loader):
    model.eval()  # 모델을 평가 모드로 설정
    correct = 0
    total = 0
    correct_per_label = {label: 0 for label in range(10)}  # 0부터 9까지의 correct 라벨 dict 초기화
    total_per_label = {label: 0 for label in range(10)}  # 0부터 9까지의 correct 라벨 누적 dict 초기화

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if isinstance(outputs, tuple):  # 훈련 모드에서 반환값 체크를 위한 조건
                outputs = outputs[0]  # 주 출력만 사용
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for label in range(10):  # 0부터 9까지의 라벨
                is_label = labels == label # 배치안에 포함된 데이터의 라벨들 == 현재 label
                total_per_label[label] += is_label.sum().item() #라벨 별 총 개수 계산
                correct_per_label[label] += (predicted[is_label] == labels[is_label]).sum().item()

    # 전체 정확도 계산
    overall_accuracy = 100 * correct / total
    print(f"Overall Accuracy: {overall_accuracy:.2f}%")

    # 라벨별 정확도 계산
    for label in range(10):
        if total_per_label[label] > 0:
            accuracy = 100 * correct_per_label[label] / total_per_label[label]
            print(f"Accuracy for label {label}: {accuracy:.2f}%")
        else:
            print(f"Label {label} does not appear in the test set.")

## 2. 전체 데이터 학습

전체 데이터의 train으로 학습 -> valid로 정확도 확인

* 기본 augmentation 데이터와 VerticalFlip(p=1) 데이터를 concat 하여 train dataset 정의한 경우
  * (기본) concat (기본의 vertical Flip) => Data 2배

* 기본 augmentation 데이터와 contour 데이터의 concat 전체를 VerticalFlip 한 것을 또 concat 하여 train dataset 정의한 경우
  * ((기본) concat (contour)) concat (앞의 vertical Flip) => Data 4배.

In [ ]:
# 모델 학습 및 평가 실행
train_model(model, train_loader, criterion, optimizer, num_epochs = num_epochs, scheduler = scheduler )
evaluate_and_calculate_accuracy(model, val_loader)

Epoch [1/200], Loss: 4.227871531052209
Epoch [2/200], Loss: 3.335070535050871
Epoch [3/200], Loss: 2.7383821782931475
Epoch [4/200], Loss: 2.517823871312567
Epoch [5/200], Loss: 2.3361581004281557
Epoch [6/200], Loss: 2.1702818204539485
Epoch [7/200], Loss: 2.029575121794508
Epoch [8/200], Loss: 1.9267479668200855
Epoch [9/200], Loss: 1.7743621068381368
Epoch [10/200], Loss: 1.6729734570767398
Epoch [11/200], Loss: 1.5675336620617362
Epoch [12/200], Loss: 1.4462330990554022
Epoch [13/200], Loss: 1.3519234911936548
Epoch [14/200], Loss: 1.2561330826069828
Epoch [15/200], Loss: 1.181160702112135
Epoch [16/200], Loss: 1.0780798893579295
Epoch [17/200], Loss: 1.0061413522617357
Epoch [18/200], Loss: 0.9655946515535525
Epoch [19/200], Loss: 0.8912227524558144
Epoch [20/200], Loss: 0.8272538788161927
Epoch [21/200], Loss: 0.7909202579881104
Epoch [22/200], Loss: 0.723372563929625
Epoch [23/200], Loss: 0.7183953760095605
Epoch [24/200], Loss: 0.6743080394928445
Epoch [25/200], Loss: 0.6238459

In [ ]:
evaluate_and_calculate_accuracy(model, val_loader)

Overall Accuracy: 94.17%
Accuracy for label 0: 100.00%
Accuracy for label 1: 85.11%
Accuracy for label 2: 88.71%
Accuracy for label 3: 96.83%
Accuracy for label 4: 98.36%
Accuracy for label 5: 96.55%
Accuracy for label 6: 86.21%
Accuracy for label 7: 97.96%
Accuracy for label 8: 92.45%
Accuracy for label 9: 98.25%


In [ ]:
evaluate_and_calculate_accuracy(model, test1_loader)

Overall Accuracy: 94.36%
Accuracy for label 0: 100.00%
Accuracy for label 1: 96.61%
Accuracy for label 2: 90.00%
Accuracy for label 3: 89.09%
Accuracy for label 4: 97.78%
Accuracy for label 5: 96.00%
Accuracy for label 6: 89.06%
Accuracy for label 7: 96.08%
Accuracy for label 8: 93.22%
Accuracy for label 9: 98.21%


In [ ]:
#모델 저장
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/path_to_pretrained_model.pt')

## 3.  라벨 3, 8, 10 만 따로 학습

valid에서 정확도가 낮은 라벨 3개를 뽑아서 재학습

#### 3.1 early stopping 클래스 정의

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.val_acc_max = 0  # 최고의 검증 정확도를 추적
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model, val_acc=None):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, val_acc, model)
        elif score < self.best_score + self.delta and (val_acc is None or val_acc < self.val_acc_max):
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            if val_acc is not None and val_acc > self.val_acc_max:
                self.save_checkpoint(val_loss, val_acc, model)
                self.best_score = score
                self.val_acc_max = val_acc
                self.counter = 0

    def save_checkpoint(self, val_loss, val_acc, model):
        if self.verbose:
            if val_acc is not None:
                print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Validation accuracy increased ({self.val_acc_max:.6f} --> {val_acc:.6f}). Saving model ...')
            else:
                print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        if val_acc is not None:
            self.val_acc_max = val_acc

# EarlyStopping 인스턴스 생성
early_stopping = EarlyStopping(patience=5, verbose=True, path='/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/best_model_fold_specific.pt')

#### 3.2 학습 및 평가 함수 정의

In [ ]:
def train_model_specific_labels(model, train_loader, val_loader, criterion, optimizer, num_epochs, scheduler, early_stopping):

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
             # GoogLeNet의 주 출력만을 사용하여 손실 계산
            if isinstance(outputs, tuple):
                outputs = outputs[0]  # GoogLeNet의 주 출력을 사용
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # 교차 검증 성능 평가를 위한 val loss 계산
        val_loss = calculate_loss_and_accuracy_specific_labels(val_loader, model, criterion)

        # Early Stopping check
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            early_stopping = EarlyStopping(patience=5, verbose=True, path='/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/best_model_fold_specific.pt')
            break

        epoch_loss = running_loss / len(train_loader)
        scheduler.step(epoch_loss)

        print(f"> > > > Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")



def calculate_loss_and_accuracy_specific_labels(loader, model, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    correct_per_label = {label: 0 for label in range(10)}
    total_per_label = {label: 0 for label in range(10)}

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if isinstance(outputs, tuple):  # 훈련 모드에서 반환값 체크를 위한 조건
              outputs = outputs[0]  # 주 출력만 사용
            _, predicted = torch.max(outputs.data, 1)

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            for label in range(10):
                is_label = labels == label
                total_per_label[label] += is_label.sum().item()
                correct_per_label[label] += (predicted[is_label] == labels[is_label]).sum().item()


    # 평균 loss계산
    average_loss = total_loss / len(loader)


    # 라벨별 정확도 계산   (특정 라벨에 대해서만 나오게함, 모델의 출력 계층을 변경 X -> 모델 구조는 그대로!)
    for label in range(10):
        if total_per_label[label] > 0:
            accuracy = 100 * correct_per_label[label] / total_per_label[label]
            print(f"Accuracy for label {label}: {accuracy:.2f}%")
        '''
        else:
            print(f"Label {label} does not appear in the test set.")
            '''

    return average_loss


#### 3.3 특정 label에 해당하는 데이터 불러오기

In [ ]:
# 특정 라벨에 해당하는 데이터셋 준비

labels_of_interest = [1, 3, 4]

specific_train_indices = [i for i, (_, label) in enumerate(train_data) if label in labels_of_interest] # 1,3,4에 해당하는 데이터 인덱스
specific_train_data = [train_data[i] for i in specific_train_indices]
specific_train_labels = [label for _, label in specific_train_data]
specific_train_dataset_original = CustomDataset(specific_train_data, transform=transform)
specific_train_dataset_filp = CustomDataset(specific_train_data, transform=flip_transform)
specific_train_dataset = torch.utils.data.ConcatDataset([specific_train_dataset_original, specific_train_dataset_filp])

specific_test_indices = [i for i, (_, label) in enumerate(test_data) if label in labels_of_interest] # 1,3,4에 해당하는 데이터 인덱스
specific_test_data = [test_data[i] for i in specific_test_indices]
specific_test_labels = [label for _, label in specific_test_data]
specific_test_dataset = CustomDataset(specific_test_data, transform=sub_transform)

In [ ]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/path_to_pretrained_model.pt')
evaluate_and_calculate_accuracy(model, val_loader)
evaluate_and_calculate_accuracy(model, test_loader)

Overall Accuracy: 93.99%
Accuracy for label 0: 100.00%
Accuracy for label 1: 89.36%
Accuracy for label 2: 90.32%
Accuracy for label 3: 95.24%
Accuracy for label 4: 96.72%
Accuracy for label 5: 96.55%
Accuracy for label 6: 81.03%
Accuracy for label 7: 100.00%
Accuracy for label 8: 92.45%
Accuracy for label 9: 98.25%
Overall Accuracy: 69.30%
Accuracy for label 0: 83.33%
Accuracy for label 1: 90.00%
Accuracy for label 2: 37.50%
Accuracy for label 3: 83.33%
Accuracy for label 4: 90.00%
Accuracy for label 5: 93.33%
Accuracy for label 6: 90.00%
Accuracy for label 7: 30.00%
Accuracy for label 8: 100.00%
Accuracy for label 9: 32.31%


In [ ]:
evaluate_and_calculate_accuracy(model, test1_loader)

Overall Accuracy: 94.89%
Accuracy for label 0: 100.00%
Accuracy for label 1: 96.61%
Accuracy for label 2: 90.00%
Accuracy for label 3: 89.09%
Accuracy for label 4: 100.00%
Accuracy for label 5: 96.00%
Accuracy for label 6: 92.19%
Accuracy for label 7: 96.08%
Accuracy for label 8: 91.53%
Accuracy for label 9: 100.00%


#### 3.4 위에서 학습시킨 모델을 불러와서 추가 10-folds 학습 실행

StratifiedKFold: 각 클래스의 샘플 비율을 유지하면서 데이터셋을 분할하는 방법
3, 8, 10 데이터의 train - valid로 10ford실행

label 0 -> 3

label 1 -> 8

label 2 -> 10

In [ ]:
# 이미 학습된 모델을 불러오는 코드 (전체 데이터셋으로 학습된 모델)
try:
  model = torch.load('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/path_to_pretrained_model.pt')
except:
  model = torch.load('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/path_to_pretrained_model.pt')


# 10-fold cross-validation 설정 # 일단 2로 작게 설정!
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 추가 학습을 위한 10-fold cross-validation 진행
for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(specific_train_labels)), specific_train_labels)):
    early_stopping = EarlyStopping(patience=5, verbose=True, path='/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/best_model_fold_specific.pt')
    print(f"Fold {fold+1}")

    # 각 fold에 대한 학습 및 검증 데이터셋 생성
    train_subset = Subset(specific_train_dataset, train_idx)
    val_subset = Subset(specific_train_dataset, val_idx)
    specific_train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
    specific_val_loader = DataLoader(val_subset, batch_size=16)

    # 학습 및 검증
    train_model_specific_labels(model, specific_train_loader, specific_val_loader, criterion, optimizer, num_epochs, scheduler, early_stopping)
    # #모델 저장
    torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/model_weights_fold_{fold+1}.pt')

specific_test_loader = DataLoader(specific_test_dataset, batch_size=16)

Fold 1
Accuracy for label 0: 92.86%
Accuracy for label 2: 81.25%
Accuracy for label 3: 100.00%
Validation loss decreased (inf --> 0.216850). Saving model ...
> > > > Epoch [1/200], Loss: 0.2855
Accuracy for label 0: 92.86%
Accuracy for label 2: 81.25%
Accuracy for label 3: 100.00%
EarlyStopping counter: 1 out of 5
> > > > Epoch [2/200], Loss: 0.3711
Accuracy for label 0: 92.86%
Accuracy for label 2: 81.25%
Accuracy for label 3: 100.00%
> > > > Epoch [3/200], Loss: 0.4105
Accuracy for label 0: 92.86%
Accuracy for label 2: 81.25%
Accuracy for label 3: 100.00%
EarlyStopping counter: 2 out of 5
> > > > Epoch [4/200], Loss: 0.3224
Accuracy for label 0: 92.86%
Accuracy for label 2: 81.25%
Accuracy for label 3: 100.00%
EarlyStopping counter: 3 out of 5
> > > > Epoch [5/200], Loss: 0.2866
Accuracy for label 0: 92.86%
Accuracy for label 2: 87.50%
Accuracy for label 3: 100.00%
EarlyStopping counter: 4 out of 5
> > > > Epoch [6/200], Loss: 0.2989
Accuracy for label 0: 100.00%
Accuracy for label 2

## 4. 최종 test data로 모델 평가 (모든 라벨에 대한 test 데이터)

ISSUE,, : class : 10 -> 3으로 바뀌면서 모델의 마지막 완전 연결 계층(fc layer)의 출력 크기가 처음 학습시킨 모델과 달라져서 문제가 발생


---> 특정 라벨만을 학습시키더라도 모델의 구조는 처음 학습시킨 때와 동일하게 유지,, 클래스 10개인데 데이터는 3개의 라벨만 넣는 형식

답변(?) : 그 어제 3,8만 학습시켰을 때도 클래스 10개로 하구 데이터만 3개 라벨 넣었었는데, 성능 괜찮아서 그대로 진행해도 될거같아용

---> 알고보니..! 기존 googlenet모델이 클래스 1000개 맞춰진건데 다시 10으로 안바꾸고 진행해서 생긴 문제였슴다. . . 10과 3이 아니라 10과 1000으로 다른거였어요 하하  

In [ ]:
try:
  model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/best_model_fold_specific.pt')) #EarlyStopping기준
except:
  model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/best_model_fold_specific.pt'))
#최종평가
evaluate_and_calculate_accuracy(model, val_loader)
evaluate_and_calculate_accuracy(model, test_loader)

Overall Accuracy: 91.34%
Accuracy for label 0: 84.48%
Accuracy for label 1: 95.74%
Accuracy for label 2: 74.19%
Accuracy for label 3: 92.06%
Accuracy for label 4: 100.00%
Accuracy for label 5: 96.55%
Accuracy for label 6: 87.93%
Accuracy for label 7: 91.84%
Accuracy for label 8: 94.34%
Accuracy for label 9: 98.25%
Overall Accuracy: 59.88%
Accuracy for label 0: 40.00%
Accuracy for label 1: 56.67%
Accuracy for label 2: 29.17%
Accuracy for label 3: 46.67%
Accuracy for label 4: 90.00%
Accuracy for label 5: 93.33%
Accuracy for label 6: 96.67%
Accuracy for label 7: 26.67%
Accuracy for label 8: 100.00%
Accuracy for label 9: 38.46%


In [ ]:
evaluate_and_calculate_accuracy(model, test1_loader)

Overall Accuracy: 91.89%
Accuracy for label 0: 91.38%
Accuracy for label 1: 100.00%
Accuracy for label 2: 81.43%
Accuracy for label 3: 83.64%
Accuracy for label 4: 100.00%
Accuracy for label 5: 96.00%
Accuracy for label 6: 92.19%
Accuracy for label 7: 88.24%
Accuracy for label 8: 94.92%
Accuracy for label 9: 94.64%


In [ ]:
try:
  model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/model_weights_fold_9.pt')) #모든 에폭 돌리고 난 후의 검정 accuarcy기준 -> 과적합 위험 있음
except:
  model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/KUBIG-DL/project/model_weights_fold_9.pt'))
#최종평가
evaluate_and_calculate_accuracy(model, specific_val_loader)
evaluate_and_calculate_accuracy(model, test_loader)

Overall Accuracy: 97.67%
Accuracy for label 0: 100.00%
Label 1 does not appear in the test set.
Accuracy for label 2: 93.33%
Accuracy for label 3: 100.00%
Label 4 does not appear in the test set.
Label 5 does not appear in the test set.
Label 6 does not appear in the test set.
Label 7 does not appear in the test set.
Label 8 does not appear in the test set.
Label 9 does not appear in the test set.
Overall Accuracy: 59.27%
Accuracy for label 0: 23.33%
Accuracy for label 1: 53.33%
Accuracy for label 2: 25.00%
Accuracy for label 3: 56.67%
Accuracy for label 4: 90.00%
Accuracy for label 5: 93.33%
Accuracy for label 6: 96.67%
Accuracy for label 7: 26.67%
Accuracy for label 8: 100.00%
Accuracy for label 9: 41.54%


In [ ]:
import shutil

# 이동할 폴더와 대상 폴더 경로 설정
source_folder = '/content/temp'
destination_folder = '/content/gdrive/MyDrive/24-1 DL KUBIG Contest/Seongmin'

# 폴더 이동
shutil.move(source_folder, destination_folder)

'/content/gdrive/MyDrive/24-1 DL KUBIG Contest/Seongmin/temp'